# Satellite tracker

This notebook make an animation of a satellite constellation from a given location (latitude, longitude coordinates)
Tles are extracted from celestrak web page
For this example, Starlink constellation are plotted seen from Yebes Observatory (Spain)

In [1]:
# need to install "ipympl" and "nodejs" packages
# then run:
# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# jupyter labextension install jupyter-matplotlib

%matplotlib widget

In [2]:
import requests
import datetime
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import ipywidgets as widgets

import cysgp4

In [4]:
layout = widgets.Layout(width='60%', height='40px') #set width and height

text = widgets.Text(
    value='https://celestrak.com/NORAD/elements/starlink.txt',
    placeholder='Paste ticket description here!',
    description='TLE-Catalogue',
    disabled=False,
    layout=layout
)
display(text)

Text(value='https://celestrak.com/NORAD/elements/starlink.txt', description='TLE-Catalogue', layout=Layout(hei…

In [5]:
#get the right TLE format
celestrak_latest = requests.get(text.value)
sat_tles = cysgp4.tles_from_text(celestrak_latest.text)

In [6]:
selected = widgets.Dropdown(
    options=["Yebes", "Effelsberg","other"],
    value="Yebes",
    description="RAS site:",
    disabled=False,
)
display(selected)


Dropdown(description='RAS site:', options=('Yebes', 'Effelsberg', 'other'), value='Yebes')

In [9]:
plt.close()
if(selected.value == 'other'):
    lonW = widgets.FloatText(
        value=0,
        description='Longitud:',
        disabled=False
    )    
    latW = widgets.FloatText(
        value=0,
        description='Latitude:',
        disabled=False
    )
    altW = widgets.FloatText(
        value=0,
        description='Altitude (m):',
        disabled=False
    )
    
    nameW = widgets.Textarea(
        value='Insert name of RAS station',
        placeholder='Type something',
        description='String:',
        disabled=False
    )
    display(nameW)
    display(lonW)
    display(latW)
    display(altW)

In [10]:
#Location
if(selected.value == 'Yebes'):
    lat = 40.523475
    lon = -3.08853
    alt = 0.915
    name_station = selected.value
elif(selected.value == 'Effelsberg'):
    lat = 50.52483
    lon = 6.88361
    alt = 0.319
    name_station = selected.value
elif(selected.value == 'other'):
    lat = latW.value
    lon = lonW.value
    alt = altW.value /1000
    name_station = nameW.value
RAS_location = cysgp4.PyObserver(lon,lat , alt)  # 13.2m TELESCOPE

In [11]:
lon,lat,alt

(6.88361, 50.52483, 0.319)

In [12]:
from threading import Thread
import time


class LiveGraph:
    def __init__(self):
        
        self.my_time = cysgp4.PyDateTime(datetime.datetime.utcnow())
        self.topo_pos_az, self.topo_pos_el, self.topo_pos_dist = [], [], []
        
        plt.close()
        self.fig = plt.figure(figsize=(8.54, 4.8), dpi=100)
        ax = self.fig.add_axes((0.1, 0.1, 0.8, 0.8))
        cax = self.fig.add_axes((0.91, 0.2, 0.02, 0.5))
        ax.set_xlabel('Azimuth [deg]')
        ax.set_ylabel('Elevation [deg]')
        ax.set_xlim((-180, 180))
        ax.set_ylim((0, 90))
        ax.set_xticks(range(-150, 180, 30))
        ax.set_yticks(range(0, 91, 30))
        # ax.set_aspect('equal')
        ax.grid()

        ax.text(-170, 75, name_station, fontsize=16)

        vmin, vmax = np.log10(100), np.log10(100000)
        self.points = ax.scatter(
            self.topo_pos_az[:], self.topo_pos_el[:],
            c=np.log10(self.topo_pos_dist[:]),
            cmap='viridis', vmin=vmin, vmax=vmax,
            )
        cbar = self.fig.colorbar(self.points, cax=cax)
        cbar.set_label('Distance (km)')
        cbar.set_ticks([2, 3, 4])
        cbar.set_ticklabels([100, 1000, 10000])
        
        self.title = ax.text(
            174, 75, '{:%y/%m/%d %H:%M:%S}'.format(self.my_time.datetime),
            fontsize=15, ha='right'
            )

        self.animation = animation.FuncAnimation(self.fig, self.update, interval=1000)
        self.th = Thread(target=self.thread_f, daemon=True)
        self.th.start()

    def update(self, frame):
        self.points.set_offsets(np.column_stack([self.topo_pos_az[:], self.topo_pos_el[:]]))
        self.points.set_array(np.log10(self.topo_pos_dist[:]))
        self.title.set_text('{:%y/%m/%d %H:%M:%S}'.format(self.my_time.datetime))
        return self.points, self.title
    
    def show(self):
        plt.show()

    def thread_f(self):
        while True:
            # get current time
            self.my_time = cysgp4.PyDateTime(datetime.datetime.utcnow())  

            result = cysgp4.propagate_many(
                self.my_time.mjd,
                sat_tles,
                RAS_location,
                do_sat_azel=False,
                )
            topo_pos = result['topo']
            #print(topo_pos)
            # topo_pos_az, topo_pos_el, topo_pos_dist, _ = (topo_pos[..., i] for i in range(4))
            self.topo_pos_az, self.topo_pos_el, self.topo_pos_dist, _ = topo_pos.T
            self.topo_pos_az = (self.topo_pos_az + 180.) % 360. - 180.
            time.sleep(1)  

In [13]:
g = LiveGraph()
g.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …